# Get Buoy and Vessel data

In [ ]:
# parameters

lit_data_url = "https://osis.geomar.de/underway/3ka2chb/api/v1/positions/Littorina2023?include_payloads=false&earliest_timestamp=2023-02-01"
lit_data_file = "lit_positions.csv"

buoy_url = "ftp.hereon.de"
buoy_path = "outgoing/carrasco/Drifters/KIEL/csv"
buoy_data_file = "buoy_positions.csv"

## Buoy positions

In [ ]:
from ftplib import FTP
from functools import partial

import pandas as pd
import hvplot.pandas

In [ ]:
def get_all_buoy_csv_file_names(url=None, path=None):
    ftp = FTP(url)
    ftp.login()
    return list(
        filter(
            lambda s: s.endswith(".csv"),
            map(
                lambda tp: f"ftp://{url}/{path}/{tp[0]}",
                ftp.mlsd(path=path)
            )
        )
    )

def load_all_buoy_csv_files(url=None, path=None):
    urls = get_all_buoy_csv_file_names(url=url, path=path)
    df = pd.concat(map(pd.read_csv, urls), ignore_index=True)
    df["date_UTC"] = pd.to_datetime(df["date_UTC"])
    df["D_number"] = df["D_number"].astype(int).astype(str)
    df = df.set_index(["D_number", "date_UTC"])
    df = df.sort_index()
    df = df.drop_duplicates()
    df = df.reset_index()
    return df

In [ ]:
df_buoys = load_all_buoy_csv_files(
    url=buoy_url,
    path=buoy_path)

df_buoys

In [ ]:
df_buoys.to_csv(buoy_data_file, index=False)
!head -n5 {buoy_data_file}

## Vessel positions

In [ ]:
import geopandas

In [ ]:
def read_littorina_positions(url=None):
    df = geopandas.read_file(
        url
    )
    
    df["Longitude"] = df.geometry.apply(lambda p: p.x)
    df["Latitude"] = df.geometry.apply(lambda p: p.y)
    df = df.drop(columns=["geometry", ])
    
    # readacted positions
    redact_here = df.datastream.str.contains("MarineTraffic")
    df["Longitude_"] = (
        redact_here * df["Longitude"].round(1)
        + ~redact_here * df["Longitude"]
    )
    df["Latitude_"] = (
        redact_here * df["Latitude"].round(1)
        + ~redact_here * df["Longitude"]
    )
    
    return df

In [ ]:
df_lit = read_littorina_positions(url=lit_data_url)

In [ ]:
df_lit.to_csv(lit_data_file, index=False)
!head -n5 {lit_data_file}